In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# !pip install -q lightning

In [1]:
import numpy as np

import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torchvision.transforms as transforms

import lightning as pl
from lightning import Trainer
from lightning.pytorch import loggers
from lightning.pytorch import callbacks


from constants import *
from dataset import ImagenesCaptchas
from models.EfficentNet import CaptchaModel_EfficentNet
from models.RestNet18 import CaptchaModel_RestNet18
from models.CRNN import CaptchaModel_ConvLSTM
from preprocessing import *
from test import *

c:\Users\pedro\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: [WinError 127] No se encontró el proceso especificado
  warn(f"Failed to load image Python extension: {e}")


In [ ]:
# PATH_TRAIN = "/content/drive/MyDrive/Datasets/data_captcha_real/train/"
# PATH_VAL = "/content/drive/MyDrive/Datasets/data_captcha_real/validation/"
print(f"Number of files in PATH_TRAIN: {len(os.listdir(PATH_TRAIN))}")
print(f"Number of files in PATH_VAL: {len(os.listdir(PATH_VAL))}")

# PATH_TRAIN_SINTETICO = "/content/drive/MyDrive/Datasets/data_captcha_sintetico/train_images/"
# PATH_VAL_SINTETICO = "/content/drive/MyDrive/Datasets/data_captcha_sintetico/validation_images/"
print(f"Number of files in PATH_TRAIN_SINTETICO: {len(os.listdir(PATH_TRAIN_SINTETICO))}")
print(f"Number of files in PATH_VAL_SINTETICO: {len(os.listdir(PATH_VAL_SINTETICO))}")

In [ ]:
files_in_train_path = os.listdir(PATH_TRAIN)[:5]
for i, file in enumerate(files_in_train_path):
    print(f"File {i+1}: {file}")

In [44]:
transform = transforms.Compose([
    transforms.Resize([256, 256], interpolation= transforms.InterpolationMode.BICUBIC),
    transforms.ToTensor(),
])

In [45]:
# creamos los datasets y los dataloaders con las imagenes reales
train_dataset = ImagenesCaptchas(PATH_TRAIN, transform)
validation_dataset = ImagenesCaptchas(PATH_VAL, transform)

train_dataloader = DataLoader(train_dataset, batch_size=32, num_workers=0, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=1, num_workers=0, shuffle=False)

# creamos los datasets y los dataloaders con las imagenes sinteticas
train_sintetic_dataset = ImagenesCaptchas(PATH_TRAIN_SINTETICO, transform)
validation_sintetic_dataset = ImagenesCaptchas(PATH_VAL_SINTETICO, transform)

train_sintetic_dataloader = DataLoader(train_sintetic_dataset, batch_size=128, num_workers=2, shuffle=True, pin_memory=False)
validation_sintetic_dataloader = DataLoader(validation_sintetic_dataset, batch_size=32, num_workers=2, pin_memory=False)

In [9]:
from torchvision.models._api import WeightsEnum
from torch.hub import load_state_dict_from_url

def get_state_dict(self, *args, **kwargs):
    kwargs.pop("check_hash")
    return load_state_dict_from_url(self.url, *args, **kwargs)
WeightsEnum.get_state_dict = get_state_dict

### entrenamiento

In [ ]:
model = CaptchaModel_EfficentNet()

In [ ]:
callback = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    verbose=False,
    mode='min'
)

logger = loggers.CSVLogger("logs", name="CSV_logs")

trainer = Trainer(max_epochs=100, fast_dev_run=False, 
                logger=logger, 
                callbacks=[callback], 
                log_every_n_steps=12, 
                accelerator = "gpu",
                devices = "auto") # callbacks=[callback]

trainer.fit(model, train_sintetic_dataloader, validation_sintetic_dataloader) # validation_dataloader


### test

In [ ]:
def evaluate_model(model, dataloader):
    model.eval()
    
    # Initialize variables to store the total correct predictions and total number of images
    total_correct = 0
    total_images = 0
    df = {"label": [], "predicted": []}

    with torch.no_grad():  # this is optional but good practice for memory efficiency
        # Iterate over the validation dataloader
        for batch in dataloader:
            images, _, labels = batch
            for image, label in zip(images, labels):
                # Perform a forward pass through the model
                predictions = model(image.unsqueeze(dim=0))
                # Split the vector per 10 elements
                predictions = predictions.view(-1, 10)
                probabilities = torch.nn.functional.softmax(predictions, dim=1)
            
                # Make an argmax to the probabilities
                _, predicted_classes = torch.max(probabilities, 1)
            
                # Get the ground truth labels
                ground_truth = label

                # Convert the predicted classes to integer tensor and join the tenso in a single number
                predicted_classes = predicted_classes.view(-1, 6).squeeze()
                str_list = [str(num.item()) for num in predicted_classes]
                result = rellenar_con_ceros(int(''.join(str_list)), 6)
            
                # Compare the predicted classes with the ground truth labels and update total correct predictions if they match
                total_correct += np.array(result == ground_truth).sum().item()
                
                df["label"].append(ground_truth)
                df["predicted"].append(result)

            total_images += len(labels)
    
    # Calculate the accuracy
    accuracy = total_correct / total_images

    print(f"Accuracy: {accuracy * 100}%")
    return df, accuracy

In [ ]:
df, accuracy = evaluate_model(model, validation_sintetic_dataloader)

In [ ]:
df.head()